In [6]:
# import json

# jsonfile = {}

# jsonObject = json.dumps(jsonfile, indent=4)

# with open('ALL_SHIP_VIA.json', 'w') as outfile:  
#     outfile.write(jsonObject)

In [7]:
# with open('testfile.json', 'r') as file:
#     data = json.load(file)
#     data["add"] = {}

In [8]:
# with open('ALL_SHIP_VIA.json', 'w') as file:
#     json.dump(data, file, indent=4)

In [9]:
# #To Append Each Time to {}
# "ordrNo" : {
#     "dateReceived" : None,
#     "pages" : None,
#     "originalPrint" : None,
#     "reprintDate" : None,
#     "emailAttachment" : None,
# }

In [10]:
import json 

def load_json(file_name):
    '''
    file_name = .json file
    '''
    with open(file_name, 'r') as read_file:
        loaded_data = json.load(read_file) # load master data
    return loaded_data

def add_to_json(file_name, key, value):
    '''
    file_name = .json file
    key = "order_no" of the target file
    value = {
                    "dateReceived" : None,
                    "originalPrint" : None,
                    "reprintDate" : None,
                    "emailAttachment" : None,
                    "fileDirectory" : None,
                    "status" : None,
                    "isExcelUpdated" : None
            }
    
    1) open json file, load data, append new data
    2) open json file, dump/save the same data to json
    
    difference 'r' and 'w'
    
    data structure to add to dictionary: 
    "ordrNo" : {
                    "dateReceived" : None,
                    "originalPrint" : None,
                    "reprintDate" : None,
                    "emailAttachment" : None,
                    "fileDirectory" : None,
                    "status" : None,
                    "isExcelUpdated" : None
                }
    "ordrNo_reprint": {
                    "dateReceived" : None,
                    "originalPrint" : None,
                    "reprintDate" : None,
                    "emailAttachment" : None,
                    "fileDirectory" : None,
                    "status" : None,
                    "isExcelUpdated" : None
                }
    '''
    with open(file_name, 'r') as read_file:
        loaded_data = json.load(read_file) # load master data
        loaded_data[key] = value # append to master data
    
    with open(file_name, 'w') as write_file:
        json.dump(loaded_data, write_file, indent=4)
        
def update_values(file_name, key, update_key, value):
    '''
    file_name = .json file
    key = to find "order_no" in hashable dict
    update_key = from one of ["dateReceived",
                    "originalPrint",
                    "reprintDate",
                    "emailAttachment",
                    "fileDirectory",
                    "status",
                    "isExcelUpdated"], also used to add new (k, v) pairs within key 
    value = update value in (update_key, value) pair
    '''
    with open(file_name, 'r') as read_file:
        loaded_data = json.load(read_file) # load master data
        loaded_data[key][update_key] = value # append to master data
    
    with open(file_name, 'w') as write_file:
        json.dump(loaded_data, write_file, indent=4)

In [11]:
from datetime import date
import os

def paths(root_path):
    '''
    root_path = takes in path where email_attachments are saved (must be r'str')
    
    all_paths = {} is a dictionary to all required directories 
    '''
    today = date.today()

    pick_ticket_path = root_path + '\\master_pick_tickets'
    email_archive = root_path + '\\email_archive'

    all_paths = {
        'root_path' : root_path,
        'pick_ticket_path' : pick_ticket_path,
        'email_archive' : email_archive,
    }
    return all_paths



def create_directory(path):
    '''
    path = directory to create (must be r'str')
    '''
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
        


def enter_directory(path):
    '''
    path = directory to enter(must be r'str')
    '''
    try: 
        os.chdir(path)
    except OSError:       
        print("Entering the directory %s failed" % path)
        
        
        
def verify_directory(path):
    '''
    path = directory to verify (must be r'str')
    '''
    return os.path.exists(path)



def move_files(src, dst):
    '''
    src = path to file (.pdf)
    dst = new path to file (.pdf)
    '''
    try:
        os.replace(src, dst)
    except:
        os.rename(src, dst)

In [12]:
import glob
import os

def list_of_files(path, file_ending='*.pdf'):
    '''
    path = directory of files
    file_ending = type of files (default = ".pdf")
    '''
    return glob.glob(os.path.join(path, file_ending))

In [13]:
from PyPDF2 import PdfFileWriter, PdfFileReader

def read_file(file, json_data, jsonfile, extraction_info, move_to):
    '''
    file = emailed file (mail%^**&^%.pdy)
    json_data = loaded json data (load everytime before calling this function)
    jsonfile = main json file ("master_pick_tickets.json")
    
    orders_dict = tracks the indices of page(s) this function has to extract from file
    json_dict_to_add = for each file json data for all picktickets is loaded only once and then appended to jsonfile
    
    data extracted form each page:
    {
        originalPrint, 
        reprintDate, 
        dateReceived, 
        emailAttachment, 
        shipTo, 
        via, 
        fileDirectory, 
        status = "Just Received", 
        isExcelUpdated = False
    }
    
    if order_no already exists in previous email attachment, then "_reprint" is appended to order_no ("7182345-00_reprint")
    '''
    orders_dict = {}
    json_dict_to_add = {}
    with open(file, 'rb') as readfile:
        input_file = PdfFileReader(readfile)
        pages = input_file.numPages

        
        for page in range(pages):
            file_writer = PdfFileWriter()
            get_page = input_file.getPage(page)
            
            reprintDate = None
            master_text = get_page.extractText()
            narrowed_text = find_pattern(r'Order(.*)FOB', master_text)
            
            order_no = find_pattern(r'(\d{4,7})[-](\d{2})', narrowed_text)
            originalPrint = True
            if if_order_as_originalPrint(order_no, json_data):
                order_no = if_order_exists_return_renamed(order_no, json_data)
                reprint_dates(order_no, jsonfile, json_dict_to_add, json_data)
                originalPrint = False
            
            dateReceived = str(datetime.now())
            emailAttachment = move_to
            
            shipTo = find_pattern(r"ShipTo(.*)Order", master_text)[8:].split(" ")[:3]
            shipTo = " ".join(shipTo)
            
            via = ship_via("ALL_SHIP_VIA.json", narrowed_text)
            
            fileDirectory = os.getcwd() + rf"\{order_no}.pdf"
            
            try:
                orders_dict[order_no].append(page)
            except:
                orders_dict[order_no] = []
                orders_dict[order_no].append(page)
            
            for each_page in orders_dict[order_no]:
                file_writer.addPage(input_file.getPage(each_page))
                
            with open(f"{order_no}.pdf", "wb") as f:
                file_writer.write(f)
                f.close()
                
            ######## Adding data
            try:
                json_dict_to_add[order_no] = append_data(originalPrint, 
                                                         reprintDate, 
                                                         dateReceived, 
                                                         emailAttachment, 
                                                         shipTo, 
                                                         via,
                                                        fileDirectory)
            except:
                print("could not add data")
                
        for k, v in json_dict_to_add.items():
            add_to_json(jsonfile, k, v)
            
        prettyInfo(file, pages, extraction_info)

In [14]:
# if order no ['original_print'] == True:
# add "_reprint"
from datetime import datetime


def reprint_dates(order_no, jsonfile, json_dict_to_verify, json_data):
    '''
    order_no = OrderNo from pickticket
    jsonfile = main json file ("master_pick_tickets.json")
    json_dict_to_verify = "json_dict_to_add" in read_file()
    json_data = loaded json data (load everytime before calling "read_file" function)
    '''
    target = json_data[order_no[:10]]['reprintDate']
    if not order_no in json_dict_to_verify:
        try:
            target.append(str(datetime.now()))
        except:
            target = []
            target.append(str(datetime.now()))
        update_values(jsonfile, order_no[:10], "reprintDate", target)
        
    

def if_order_as_originalPrint(order_no, json_data):
    '''
    order_no = OrderNo from pickticket
    json_data = loaded json data (load everytime before calling "read_file()" function)
    '''
    if order_no in json_data:
        return json_data[order_no]["originalPrint"]

def if_order_exists_return_renamed(order_no, json_data):
    '''
    order_no = OrderNo from pickticket
    json_data = loaded json data (load everytime before calling "read_file()" function)
    '''
    while order_no in json_data:
        order_no = order_no + "_reprint"
    return order_no 

def append_data(originalPrint, reprintDate, dateReceived, 
                emailAttachment, shipTo, via, fileDirectory, 
                status="Just Received", isExcelUpdated=False):
    '''
    all mandatory data
    '''
    return_dict = {
                    "dateReceived" : dateReceived,
                    "originalPrint" : originalPrint,
                    "reprintDate" : reprintDate,
                    "emailAttachment" : emailAttachment,
                    "shipTo" : shipTo,
                    "via": via,
                    "fileDirectory" : fileDirectory,
                    "status" : status,
                    "isExcelUpdated" : isExcelUpdated,
                    }
    return return_dict

def ship_via(jsonfile, narrowed_text):
    '''
    jsonfile = main json file ("ALL_SHIP_VIA.json")
    narrowed_text = is narrowed_text in read_file() function
    
    matches "via" with SX data for "ship via"
    '''
    via_json = load_json(jsonfile)
    via_return = "NOT FOUND"
    for via_name in via_json["Signode_Ship_Via"]:
        if via_name in narrowed_text:
            via_return = via_name
    return via_return

In [15]:
try:
    import re2 as re
except ImportError:
    import re

def find_pattern(pattern_to_search_for, data):
    '''
    pattern_to_search_for = pattern match (must be r'str') 
    data = find pattern in data
    '''
    pattern = re.compile(pattern_to_search_for)
    pattern_matches = pattern.finditer(data)
    initial_match = [match for match in pattern_matches]
    return initial_match[0].group(0)

In [16]:
from prettytable import PrettyTable

def table():
    extraction_info = PrettyTable()
    extraction_info.field_names = ["pdf file name", "num of pages extracted"]
    return extraction_info

def prettyInfo(file, pages, extraction_info):
    '''
    file = emailed file (mail%^**&^%.pdy)
    pages = pages in file
    '''
    extraction_info.add_row([os.path.basename(file), pages])
    
def print_to_console(extraction_info):
    '''
    info = extraction_info
    '''
    print(extraction_info)

In [17]:
# import pandas as pd
# os.chdir(r"C:\Users\0235124\OneDrive - University of Waterloo\Desktop\signodeProjects\pdfParse\pyPDF2\python_code\STABLE CODE\Jupyter")

# df = pd.read_json("testfile.json")
# df = df.transpose()
# df.to_excel("op1.xlsx")


In [18]:
# from openpyxl import Workbook

# os.chdir(r"C:\Users\0235124\OneDrive - University of Waterloo\Desktop\signodeProjects\pdfParse\pyPDF2\python_code\STABLE CODE\Jupyter")

# workbook = Workbook()
# sheet = workbook.active

# sheet["A1"] = "hello"
# sheet["B1"] = "world!"

# workbook.save(filename="hello_world.xlsx")


In [19]:
# from .json import load_json, add_to_json, update_values 
# from .path import paths, create_directory, enter_directory, verify_directory, move_files
# from .algorithm import read_file
# from .algoHelpers import reprint_dates, if_order_as_originalPrint, if_order_exists_return_renamed, append_data, ship_via
# from .pattern import find_pattern
# from .prettytable import prettyInfo, print_to_console
# from allFiles import list_of_files

In [126]:
temp_path = paths(r"C:\Users\0235124\OneDrive - University of Waterloo\Desktop\signodeProjects\pdfParse\pyPDF2\python_code\STABLE CODE\test")
code_path = r"C:\Users\0235124\OneDrive - University of Waterloo\Desktop\signodeProjects\pdfParse\pyPDF2\python_code\STABLE CODE\main"

if os.getcwd() != temp_path["root_path"]:
    enter_directory(temp_path["root_path"])

if not (verify_directory(temp_path["pick_ticket_path"]) and verify_directory(temp_path["email_archive"])):
    create_directory(temp_path["pick_ticket_path"])
    create_directory(temp_path["email_archive"])
    
extraction_info = table()

jsonfile = r"C:\Users\0235124\OneDrive - University of Waterloo\Desktop\signodeProjects\pdfParse\pyPDF2\python_code\STABLE CODE\test\master_pick_tickets\master_pick_tickets.json"

all_files = list_of_files(temp_path["root_path"])

for file in all_files:
    json_data = load_json(jsonfile)
    move_to = temp_path["email_archive"] + rf"\{os.path.basename(file)}"
    
    if os.getcwd() != temp_path["pick_ticket_path"]:
        enter_directory(temp_path["pick_ticket_path"])
        
    read_file(file, json_data, jsonfile, extraction_info, move_to)
    
    if os.getcwd() != temp_path["root_path"]:
        enter_directory(temp_path["root_path"])
    
    move_files(file, move_to)
    
print_to_console(extraction_info)


if os.getcwd() != code_path:
    enter_directory(code_path)

+-------------------+------------------------+
|   pdf file name   | num of pages extracted |
+-------------------+------------------------+
| single_pdfs_5.pdf |           5            |
+-------------------+------------------------+


In [190]:
# temp_path = paths(r"C:\Users\0235124\OneDrive - University of Waterloo\Desktop\signodeProjects\pdfParse\pyPDF2\python_code\STABLE CODE\test")
# print(temp_path)

In [191]:
import os
os.chdir(r"C:\Users\0235124\OneDrive - University of Waterloo\Desktop\signodeProjects\pdfParse\pyPDF2\python_code\STABLE CODE\test\master_pick_tickets")

In [192]:
import win32com.client as win

active_exl = win.GetActiveObject('Excel.Application')

active_wrkbk = active_exl.Workbooks(1)
active_wrksht = active_wrkbk.Worksheets(1)

In [193]:
file_name = r"C:\Users\0235124\OneDrive - University of Waterloo\Desktop\signodeProjects\pdfParse\pyPDF2\python_code\STABLE CODE\test\master_pick_tickets\master_pick_tickets.json"
temp_json = load_json(file_name)

In [194]:
temp_dict = {
        "orderNo" : "A",
        "status" : "B",
        "shipTo" : "C",
        "via" : "D",
        "dateReceived" : "E",
        "friendly_name" : "F",
        "fileDirectory" : "G",
        "emailAttachment" : "H",
        "rePrint" : "I",
        }

In [195]:
temp_lst = [
        "orderNo",
        "status",
        "shipTo",
        "via",
        "dateReceived",
        "friendly_name",
        "fileDirectory",
        "emailAttachment",
        "rePrint",
]

In [179]:
def lst_unupdated_exl(json_data):
    list_return = []
    for k, v in json_data.items():
        if v["isExcelUpdated"] == False:
            list_return.append(k)
    return list_return

In [180]:
c = 2
while active_wrksht.Range(f"{temp_dict['orderNo']}{c}").Value != 0:
    update_values(file_name, 
                  active_wrksht.Range(f"{temp_dict['orderNo']}{c}").Value, 
                  "status", 
                  active_wrksht.Range(f"{temp_dict['status']}{c}").Value)
    c += 1
else:
    for order in lst_unupdated_exl(temp_json):
        update_values(file_name, order, "isExcelUpdated", True)
        for i in temp_lst[1:]:
            try:
                active_wrksht.Range(f"{temp_dict[i]}{c}").Value = temp_json[order][i]
            except:
                if i == "friendly_name":
                    active_wrksht.Range(f"{temp_dict[i]}{c}").Value = order
                elif i == "rePrint":
                    active_wrksht.Range(f"{temp_dict[i]}{c}").Value = temp_json[order]["reprintDate"]
        c += 1

In [147]:
temp_json = load_json(file_name)
temp_json

{'7182119-00': {'dateReceived': '2021-02-12 14:45:01.466045',
  'originalPrint': True,
  'reprintDate': None,
  'emailAttachment': 'C:\\Users\\0235124\\OneDrive - University of Waterloo\\Desktop\\signodeProjects\\pdfParse\\pyPDF2\\python_code\\STABLE CODE\\test\\email_archive\\single_pdfs_5.pdf',
  'shipTo': 'Advance Shipping SuppliesUnit',
  'via': 'Pick Up',
  'fileDirectory': 'C:\\Users\\0235124\\OneDrive - University of Waterloo\\Desktop\\signodeProjects\\pdfParse\\pyPDF2\\python_code\\STABLE CODE\\test\\master_pick_tickets\\7182119-00.pdf',
  'status': 'Just Received',
  'isExcelUpdated': False},
 '7177969-00': {'dateReceived': '2021-02-12 14:45:01.507465',
  'originalPrint': True,
  'reprintDate': None,
  'emailAttachment': 'C:\\Users\\0235124\\OneDrive - University of Waterloo\\Desktop\\signodeProjects\\pdfParse\\pyPDF2\\python_code\\STABLE CODE\\test\\email_archive\\single_pdfs_5.pdf',
  'shipTo': 'Rio Tinto Alcan',
  'via': 'See Below',
  'fileDirectory': 'C:\\Users\\0235124\\

In [124]:
# active_wrksht.Range("I2").Value = str(len(temp_json["7182119-00"]["reprintDate"]))

In [111]:
# temp_json["7182119-00"]["reprintDate"]

['1', '2']

In [123]:
# print(type(str(len(temp_json["7182119-00"]["reprintDate"]))))

<class 'str'>


In [125]:
# lest = "7182345-00_reprint_reprint"
# lest.split('_')

['7182345-00', 'reprint', 'reprint']

In [196]:
c = 2

def lst_unupdated_exl(json_data):
    list_return = []
    for k, v in json_data.items():
        if v["isExcelUpdated"] == False:
            list_return.append(k)
    return list_return

def return_emty_cell(c):
    while active_wrksht.Range(f"{temp_dict['orderNo']}{c}").Value != 0:
        update_values(file_name, 
                      active_wrksht.Range(f"{temp_dict['orderNo']}{c}").Value, 
                      "status", 
                      active_wrksht.Range(f"{temp_dict['status']}{c}").Value)
        c += 1
    return c

def fill_cell(temp_lst, temp_dict, c):
    for i in temp_lst[1:]:
        try:
            active_wrksht.Range(f"{temp_dict[i]}{c}").Value = temp_json[order][i]
        except:
            if i == "friendly_name":
                active_wrksht.Range(f"{temp_dict[i]}{c}").Value = order
            elif i == "rePrint":
                active_wrksht.Range(f"{temp_dict[i]}{c}").Value = temp_json[order]["reprintDate"]

c = return_emty_cell(c)

for order in lst_unupdated_exl(temp_json):
    update_values(file_name, order, "isExcelUpdated", True)
    fill_cell(temp_lst, temp_dict, c)
    c = return_emty_cell(c)